In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/spam.csv")

In [ ]:
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
X = df.drop("Category", axis=1)
y = df["Category"]

In [ ]:
X.head(5)

,Message
0,"Go until jurong point, crazy.. Available only ..."
1,Ok lar... Joking wif u oni...
2,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...
4,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
y.head(5)

,Category
0,ham
1,ham
2,spam
3,ham
4,ham


In [ ]:
texts = df["Message"][:10].tolist()

In [ ]:
def preprocess_text(text):
    text = text.lower()  # Lowercase the text
    text = text.strip()  # Remove leading/trailing whitespace
    return text

texts = [
    preprocess_text("Click here to claim your free prize!"),
    preprocess_text("Hey, are you free to chat later?")
]

inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")


In [ ]:
for text in texts:

  print(text)

  def preprocess_text(text):
    text = text.lower()  # Lowercase the text
    text = text.strip()  # Remove leading/trailing whitespace
    return text

  text = preprocess_text(text)
  print(text)

  # Preprocess the texts
  inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

  # Perform classification
  with torch.no_grad():
      outputs = model(**inputs)

  # Convert logits to probabilities
  probs = F.softmax(outputs.logits, dim=-1)

  # Interpret results
  labels = ["Not Spam", "Spam"]  # Example labels; adjust based on the model's labeling
  for text, prob in zip(text, probs):
      predicted_class = torch.argmax(prob).item()
      print(f"Text: {text}")
      print(f"Classification: {labels[predicted_class]}, Confidence: {prob[predicted_class]:.2f}\n")

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
go until jurong point, crazy.. available only in bugis n great world la e buffet... cine there got amore wat...
Text: g
Classification: Spam, Confidence: 1.00

Ok lar... Joking wif u oni...
ok lar... joking wif u oni...
Text: o
Classification: Spam, Confidence: 0.99

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005. text fa to 87121 to receive entry question(std txt rate)t&c's apply 08452810075over18's
Text: f
Classification: Not Spam, Confidence: 1.00

U dun say so early hor... U c already then say...
u dun say so early hor... u c already then say...
Text: u
Classification: Not Spam, Confidence: 0.99

Nah I don't think he goes to usf, he lives around here though
nah i don't think he goes to usf, he lives

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
import numpy as np
import evaluate

# Step 1: Load Kaggle Dataset
dataset_path = "/content/spam.csv"  # Update this with your dataset file path
df = pd.read_csv(dataset_path, encoding="ISO-8859-1")

# Preview the dataset
print("Dataset Preview:")
print(df.head())

Dataset Preview:
  Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...


In [ ]:
# Rename Columns and Prepare Data
df = df.rename(columns={"Category": "label", "Message": "text"})
df = df[["label", "text"]]

# Map labels to numeric values
label_map = {"ham": 0, "spam": 1}
df["label"] = df["label"].map(label_map)

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)

# Split into train and test datasets
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("cybert79/spamai")

# Tokenization Function
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# Tokenize Dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Map:   0%|          | 0/4457 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

In [ ]:
# Load Model
model = AutoModelForSequenceClassification.from_pretrained("cybert79/spamai", num_labels=2)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# Define Metrics
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
# Define Training Arguments
training_args = TrainingArguments(
    output_dir="/content/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Fine-Tune the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-11-e1ccc057b32f>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.063170,0.983857
2,0.063900,0.038250,0.993722
3,0.063900,0.039671,0.992825


TrainOutput(global_step=837, training_loss=0.041206278396478835, metrics={'train_runtime': 446.8088, 'train_samples_per_second': 29.926, 'train_steps_per_second': 1.873, 'total_flos': 879514480304640.0, 'train_loss': 0.041206278396478835, 'epoch': 3.0})

In [ ]:
# Save the Fine-Tuned Model
model.save_pretrained("/content/fine_tuned_spam_model")
tokenizer.save_pretrained("/content/fine_tuned_spam_model")


('/content/fine_tuned_spam_model/tokenizer_config.json',
 '/content/fine_tuned_spam_model/special_tokens_map.json',
 '/content/fine_tuned_spam_model/vocab.txt',
 '/content/fine_tuned_spam_model/added_tokens.json',
 '/content/fine_tuned_spam_model/tokenizer.json')

In [ ]:
# Evaluate the Model
results = trainer.evaluate()
print(f"Evaluation Results: {results}")

Evaluation Results: {'eval_loss': 0.03967087343335152, 'eval_accuracy': 0.9928251121076234, 'eval_runtime': 7.5978, 'eval_samples_per_second': 146.753, 'eval_steps_per_second': 18.426, 'epoch': 3.0}


In [ ]:
import torch

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Move input tensors to the same device
texts = ["You won a lottery! Claim your prize now!", "Hello, just checking in for our meeting."]
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)

# Forward pass
outputs = model(**inputs)

# Calculate probabilities
probs = torch.softmax(outputs.logits, dim=-1)
print(probs)


tensor([[5.6944e-05, 9.9994e-01],
        [9.9992e-01, 8.1948e-05]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [ ]:
  # Interpret results
  labels = ["Not Spam", "Spam"]
  for text, prob in zip(texts, probs):
      predicted_class = torch.argmax(prob).item()
      print(f"Text: {text}")
      print(f"Classification: {labels[predicted_class]}, Confidence: {prob[predicted_class]:.2f}\n")

Text: You won a lottery! Claim your prize now!
Classification: Spam, Confidence: 1.00

Text: Hello, just checking in for our meeting.
Classification: Not Spam, Confidence: 1.00



In [ ]:
df_1 = pd.read_csv("/content/spam.csv")

In [ ]:
df_1.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
X = df_1.drop("Category", axis=1)
y = df_1["Category"]

In [ ]:
y.head(10)

,Category
0,ham
1,ham
2,spam
3,ham
4,ham
5,spam
6,ham
7,ham
8,spam
9,spam


In [ ]:
texts = df_1["Message"][:10].tolist()

In [ ]:
import torch
import torch.nn.functional as F

# Assume `model` and `tokenizer` are already defined and loaded.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move the model to the device

# Iterate through each text
for text in texts:
    # print(f"Original Text: {text}")

    # Define a text preprocessing function
    def preprocess_text(text):
        text = text.lower()  # Lowercase the text
        text = text.strip()  # Remove leading/trailing whitespace
        return text

    # Preprocess the text
    text = preprocess_text(text)

    # Tokenize and move input tensors to the same device as the model
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device)

    # Perform classification
    with torch.no_grad():
        outputs = model(**inputs)

    # Convert logits to probabilities
    probs = F.softmax(outputs.logits, dim=-1)

    # Interpret results
    labels = ["Not Spam", "Spam"]  # Example labels; adjust based on the model's labeling
    predicted_class = torch.argmax(probs, dim=-1).item()
    confidence = probs[0, predicted_class].item()

    # Output the classification and confidence
    print(f"Processed Text: {text}")
    print(f"Classification: {labels[predicted_class]}, Confidence: {confidence:.2f}\n")


Processed Text: go until jurong point, crazy.. available only in bugis n great world la e buffet... cine there got amore wat...
Classification: Not Spam, Confidence: 1.00

Processed Text: ok lar... joking wif u oni...
Classification: Not Spam, Confidence: 1.00

Processed Text: free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005. text fa to 87121 to receive entry question(std txt rate)t&c's apply 08452810075over18's
Classification: Spam, Confidence: 1.00

Processed Text: u dun say so early hor... u c already then say...
Classification: Not Spam, Confidence: 1.00

Processed Text: nah i don't think he goes to usf, he lives around here though
Classification: Not Spam, Confidence: 1.00

Processed Text: freemsg hey there darling it's been 3 week's now and no word back! i'd like some fun you up for it still? tb ok! xxx std chgs to send, £1.50 to rcv
Classification: Spam, Confidence: 1.00

Processed Text: even my brother is not like to speak with me. they treat me like aids pate